In [1]:
!pip install spacy

It turns out that spaCy requires a "model" file, which is a bundle of statistical information that allows the library to parse text into words and parts of speech. While spaCy comes with a model when you install it, that model does *not* include word vectors, so you'll need to download a model that does include them. For English, I recommend `en_core_web_lg`, which you can download and install by running the cell below. (The model file is fairly large and might take a while to download.)

In [3]:
!python -m spacy download en_core_web_lg

[+] Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


Once download is complete, <b>restart the colab runtime </b>(to do this in the colab menu, go for Runtime > Restart runtime).<br>
The code in the following cell loads `spacy` and the model you just downloaded:

In [ ]:
import spacy
import en_core_web_lg
nlp = spacy.load('en_core_web_lg')

nlp = en_core_web_lg.load()

In [7]:
nlp.vocab['amazing'].vector # replace cheese with whatever word you want!

array([-2.6565e-01,  4.6795e-01,  5.8068e-02,  2.9165e-03, -1.3532e-01,
        2.1199e-01,  8.2157e-02, -6.2320e-02, -5.3217e-02,  1.8894e+00,
        1.6439e-01,  3.1966e-01, -9.3268e-02,  1.4703e-01,  1.0479e-01,
       -1.7459e-01, -4.4935e-01,  1.2442e+00, -1.9939e-01, -3.3135e-01,
        3.1638e-01,  9.2198e-03,  2.1116e-01, -3.8205e-01, -1.1604e-01,
        2.6201e-01,  2.7354e-01, -6.7279e-02, -1.9275e-01,  2.3613e-02,
        2.2156e-01,  2.3550e-01,  2.6286e-02,  2.5266e-01,  2.8450e-03,
       -2.5412e-01,  2.5801e-01, -2.1384e-01, -1.5835e-01, -1.4999e-01,
        1.8592e-01, -4.6072e-02, -7.5218e-02,  3.1008e-01,  1.3776e-01,
        5.6249e-01, -4.1056e-01,  5.7491e-01,  3.9835e-01, -5.5174e-02,
       -3.5544e-02,  2.0639e-02,  1.0778e-01, -4.0423e-02,  1.5546e-01,
        5.8229e-02,  4.5573e-02,  1.5203e-01,  2.2158e-01,  1.4574e-01,
        5.0506e-03, -4.5902e-01, -7.5091e-01,  1.2658e-01, -1.6015e-01,
       -3.6611e-01,  1.1662e-01,  2.1730e-01,  4.2705e-01,  2.23

You can look up the word vector for a particular word using spaCy right out the box like so:

It might not look much, but that list of three hundred numbers is spaCy's idea of what "cheese" means.

### Parsing a corpus of conversations

So now we need some data for the bot. In particular, we need some conversations: the text of the turns along with information about which turn is in response to which. Fortunately, some researchers at Cornell University have made available a very interesting corpus of conversations: [The Cornell Movie Dialog Corpus](https://www.cs.cornell.edu/~cristian/Cornell_Movie-Dialogs_Corpus.html), containing "220,579 conversational exchanges between 10,292 pairs of movie characters." Very cool. The data is stored in several plain text files, which you can download by running the following cells:

In [ ]:
!curl -L -O http://www.cs.cornell.edu/~cristian/data/cornell_movie_dialogs_corpus.zip

In [ ]:
!unzip cornell_movie_dialogs_corpus.zip

We'll be working with two files from this corpus. One file (`movie_lines.txt`) has the movie lines themselves, associated with a short unique identifier; another file (`movie_conversations.txt`) has lists of which lines occurred together in conversations, in the order in which they occurred. The following two cells parse these two files and create lookup dictionaries that associate unique IDs to lines (`movie_lines`) and each line to the line that follows it (`responses`).

In [ ]:
movie_lines = {}
for line in open("./cornell movie-dialogs corpus/movie_lines.txt",
                 encoding="latin1"):
    line = line.strip()
    parts = line.split(" +++$+++ ")
    if len(parts) == 5:
        movie_lines[parts[0]] = parts[4]
    else:
        movie_lines[parts[0]] = ""

In [2]:
import json

In [ ]:
import json
responses = {}
for line in open("./cornell movie-dialogs corpus/movie_conversations.txt",
                 encoding="latin1"):
    line = line.strip()
    parts = line.split(" +++$+++ ")
    line_ids = json.loads(parts[3].replace("'", '"'))
    for first, second in zip(line_ids[:-1], line_ids[1:]):
        responses[first] = second

Just to make sure everything works, the cell below prints out five random pairs of conversational turns from the corpus:

In [ ]:
import random
for pair in random.sample(responses.items(), 5):
    print("A:", movie_lines[pair[0]])
    print("B:", movie_lines[pair[1]])
    print()

### Making a sentence vector

To make the sentence vector for each line of dialog, i am using spaCy. The function `sentence_mean` below takes the spaCy object that we loaded earlier (`nlp`) and uses it to tokenize the string that you pass into the function (i.e., break it up into words). It then uses numpy's `mean()` function to find the average of the vectors, producing a new vector. The shape of the resulting vector (i.e., the number of dimensions) should be the same as the shape of the individual word vectors.

(Note: I disabled the `tagger` and `parser` parts of spaCy's pipeline to improve performance. We're not using part of speech tags or dependency relations in this chatbot, so there's no reason to spend time calculating them.)

In [ ]:
import numpy as np
def sentence_mean(nlp, s):
    if s == "":
        s = " "
    doc = nlp(s, disable=['tagger', 'parser'])
    return np.mean(np.array([w.vector for w in doc]), axis=0)
sentence_mean(nlp, "This... is a test.").shape

### Similarity lookups



In [ ]:
!pip install simpleneighbors

In [ ]:
from simpleneighbors import SimpleNeighbors

nns = SimpleNeighbors(300)
for i, line_id in enumerate(random.sample(list(responses.keys()), 10000)):
    # show progress
    if i % 1000 == 0: print(i, line_id, movie_lines[line_id])
    line_text = movie_lines[line_id]
    summary_vector = sentence_mean(nlp, line_text)
    if np.any(summary_vector):
        nns.add_one(line_id, summary_vector)
nns.build()

Let's take it for a spin! The code in the following cell finds the turn most similar to the string in the variable `sentence`. (You can change this string to whatever you want.) It then uses the Simple Neighbors object to find the turn in the database with the most similar vector, and then uses the `responses` lookup to find the *response* to that turn. That response will be our bot's output.

In [ ]:
sentence = "I like making bots."
picked = nns.nearest(sentence_mean(nlp, sentence), 5)[0]
response_line_id = responses[picked]

print("Your line:\n\t", sentence)
print("Most similar turn:\n\t", movie_lines[picked])
print("Response to most similar turn:\n\t", movie_lines[response_line_id])

In [ ]:
!pip install https://github.com/aparrish/semanticsimilaritychatbot/archive/master.zip

Then create a chatbot object, passing in the spaCy language object (`nlp`) and the number of dimensions:

In [ ]:
from semanticsimilaritychatbot import SemanticSimilarityChatbot
chatbot = SemanticSimilarityChatbot(nlp, 300)

The `.add_pair()` method in the object takes two strings: a turn and the response to that turn. We'll get these from the `responses` and `movie_lines` lookups, again sampling ten thousand pairs at random. This cell will take a little while to run:

In [ ]:
sample_n = 10000
for first_id, second_id in random.sample(list(responses.items()), sample_n):
    chatbot.add_pair(movie_lines[first_id], movie_lines[second_id])
chatbot.build()

In [ ]:
print(chatbot.response_for("Hello computer!"))

In [ ]:
my_turn = "The weather's nice today, don't you think?"
for i in range(5, 51, 5):
    print("picking from", i, "possible responses:")
    print(chatbot.response_for(my_turn, i))
    print()

The Semantic Similarity Chatbot object has a `.save()` method that saves the pre-built database to disk, using a filename prefix you supply. (It saves three different files: `<prefix>.annoy`, `<prefix>-data.pkl`, and `<prefix>-chatbot.pkl`).

In [ ]:
chatbot.save("movielines-10k-sample")

In [ ]:
chatbot = SemanticSimilarityChatbot.load("movielines-10k-sample", nlp)

In [ ]:
print(chatbot.response_for("I'm going to go get some coffee."))

In [ ]:
from google.colab import files
files.download('movielines-10k-sample.annoy')
files.download('movielines-10k-sample-data.pkl')
files.download('movielines-10k-sample-chatbot.pkl')

## Making it interactive

If you're using this notebook in Google Colab, the following cell will create a little interactive interface for chatting with the bot that you just built. Run the two cells below and start typing into the box.

In [ ]:
chatbot_html = """
<style type="text/css">#log p { margin: 5px; font-family: sans-serif; }</style>
<div id="log"
     style="box-sizing: border-box;
            width: 600px;
            height: 32em;
            border: 1px grey solid;
            padding: 2px;
            overflow: scroll;">
</div>
<input type="text" id="typehere" placeholder="type here!"
       style="box-sizing: border-box;
              width: 600px;
              margin-top: 5px;">
<script>
function paraWithText(t) {
    let tn = document.createTextNode(t);
    let ptag = document.createElement('p');
    ptag.appendChild(tn);
    return ptag;
}
document.querySelector('#typehere').onchange = async function() {
    let inputField = document.querySelector('#typehere');
    let val = inputField.value;
    inputField.value = "";
    let resp = await getResp(val);
    let objDiv = document.getElementById("log");
    objDiv.appendChild(paraWithText('😀: ' + val));
    objDiv.appendChild(paraWithText('🤖: ' + resp));
    objDiv.scrollTop = objDiv.scrollHeight;
};
async function colabGetResp(val) {
    let resp = await google.colab.kernel.invokeFunction(
        'notebook.get_response', [val], {});
    return resp.data['application/json']['result'];
}
async function webGetResp(val) {
    let resp = await fetch("/response.json?sentence=" + 
        encodeURIComponent(val));
    let data = await resp.json();
    return data['result'];
}
</script>
"""

In [ ]:
 import IPython
from google.colab import output

display(IPython.display.HTML(chatbot_html + \
                             "<script>let getResp = colabGetResp;</script>"))

def get_response(val):
    resp = chatbot.response_for(val)
    return IPython.display.JSON({'result': resp})

output.register_callback('notebook.get_response', get_response)

If you're not using Colab, try the following two cells to install [Flask](http://flask.pocoo.org) and run a little web server from your notebook that lets you chat with the bot. Click on the link that appears below the second cell to open up the chat in a new window.

In [ ]:
!pip install flask

In [ ]:
from flask import Flask, request, jsonify
app = Flask(__name__)
@app.route("/response.json")
def response():
    sentence = request.args['sentence']
    return jsonify(
        {'result': chatbot.response_for(sentence)})
@app.route("/")
def home():
    return chatbot_html + "<script>let getResp = webGetResp;</script>"
app.run()